Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

**Add L2 loss to a logistic model**

In [21]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + \
    0.01 * (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [22]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("\nMinibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized

Minibatch loss at step 0: 45.555367
Minibatch accuracy: 7.8%
Validation accuracy: 16.6%

Minibatch loss at step 500: 1.026521
Minibatch accuracy: 79.7%
Validation accuracy: 80.7%

Minibatch loss at step 1000: 0.656263
Minibatch accuracy: 84.4%
Validation accuracy: 80.6%

Minibatch loss at step 1500: 0.896476
Minibatch accuracy: 80.5%
Validation accuracy: 80.3%

Minibatch loss at step 2000: 0.792826
Minibatch accuracy: 80.5%
Validation accuracy: 81.0%

Minibatch loss at step 2500: 0.693669
Minibatch accuracy: 82.8%
Validation accuracy: 80.2%

Minibatch loss at step 3000: 0.696768
Minibatch accuracy: 85.2%
Validation accuracy: 81.7%
Test accuracy: 88.7%


**Add L2 loss to a neural network model**

In [25]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Hidden layer
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  hidden_biases = tf.Variable(tf.zeros([1024]))

  hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(hidden_layer, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + \
    0.01 * (tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(hidden_biases) +
            tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases), weights) + biases)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases), weights) + biases)

In [26]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("\nMinibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized

Minibatch loss at step 0: 3455.434082
Minibatch accuracy: 7.0%
Validation accuracy: 36.9%

Minibatch loss at step 500: 21.311453
Minibatch accuracy: 85.2%
Validation accuracy: 84.4%

Minibatch loss at step 1000: 0.827859
Minibatch accuracy: 85.9%
Validation accuracy: 82.8%

Minibatch loss at step 1500: 0.789544
Minibatch accuracy: 83.6%
Validation accuracy: 82.4%

Minibatch loss at step 2000: 0.751156
Minibatch accuracy: 82.8%
Validation accuracy: 83.2%

Minibatch loss at step 2500: 0.694164
Minibatch accuracy: 85.9%
Validation accuracy: 83.0%

Minibatch loss at step 3000: 0.677404
Minibatch accuracy: 85.9%
Validation accuracy: 84.0%
Test accuracy: 90.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [34]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Hidden layer
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  hidden_biases = tf.Variable(tf.zeros([1024]))

  hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(hidden_layer, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases), weights) + biases)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases), weights) + biases)

In [35]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % 10000
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("\nMinibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized

Minibatch loss at step 0: 329.965393
Minibatch accuracy: 12.5%
Validation accuracy: 33.7%

Minibatch loss at step 500: 18.953644
Minibatch accuracy: 89.8%
Validation accuracy: 79.2%

Minibatch loss at step 1000: 3.086717
Minibatch accuracy: 96.9%
Validation accuracy: 81.3%

Minibatch loss at step 1500: 2.930993
Minibatch accuracy: 97.7%
Validation accuracy: 82.2%

Minibatch loss at step 2000: 0.028172
Minibatch accuracy: 98.4%
Validation accuracy: 82.3%

Minibatch loss at step 2500: 0.628493
Minibatch accuracy: 98.4%
Validation accuracy: 82.1%

Minibatch loss at step 3000: 0.112169
Minibatch accuracy: 99.2%
Validation accuracy: 82.5%
Test accuracy: 89.2%


The neural network memorizes the training set and performs poorly on the validation set!

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [36]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Hidden layer
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  hidden_biases = tf.Variable(tf.zeros([1024]))

  hidden_layer = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases), 0.5)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.nn.dropout(tf.matmul(hidden_layer, weights) + biases, 0.5)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases), weights) + biases)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases), weights) + biases)

In [37]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % 10000
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("\nMinibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized

Minibatch loss at step 0: 759.539001
Minibatch accuracy: 13.3%
Validation accuracy: 18.9%

Minibatch loss at step 500: 65.236465
Minibatch accuracy: 48.4%
Validation accuracy: 80.8%

Minibatch loss at step 1000: 24.334694
Minibatch accuracy: 48.4%
Validation accuracy: 80.0%

Minibatch loss at step 1500: 36.486465
Minibatch accuracy: 48.4%
Validation accuracy: 80.2%

Minibatch loss at step 2000: 7.372968
Minibatch accuracy: 46.9%
Validation accuracy: 79.7%

Minibatch loss at step 2500: 18.245176
Minibatch accuracy: 50.0%
Validation accuracy: 80.3%

Minibatch loss at step 3000: 14.671097
Minibatch accuracy: 51.6%
Validation accuracy: 80.0%
Test accuracy: 86.6%


It no longer overfits!

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [86]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Hidden layer 1
    hidden_weights1 = tf.Variable(
      tf.truncated_normal([image_size * image_size, 1024], stddev=0.1))
    hidden_biases1 = tf.Variable(tf.constant(0.1, shape=[1024]))

    hidden_layer1 = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights1) + hidden_biases1), 0.75)
  

    # Hidden layer 2
    hidden_weights2 = tf.Variable(
      tf.truncated_normal([1024, 2048], stddev=0.1))
    hidden_biases2 = tf.Variable(tf.constant(0.1, shape=[2048]))

    hidden_layer2 = tf.nn.dropout(tf.nn.relu(tf.matmul(hidden_layer1, hidden_weights2) + hidden_biases2), 0.75)

    # Hidden layer 3
    hidden_weights3 = tf.Variable(
      tf.truncated_normal([2048, 1024], stddev=0.1))
    hidden_biases3 = tf.Variable(tf.constant(0.1, shape=[1024]))

    hidden_layer3 = tf.nn.dropout(tf.nn.relu(tf.matmul(hidden_layer2, hidden_weights3) + hidden_biases3), 0.75)
  
    # Variables.
    weights = tf.Variable(
      tf.truncated_normal([1024, num_labels], stddev=0.1))
    biases = tf.Variable(tf.constant(0.1, shape=[num_labels]))
  
    # Training computation.
    logits = tf.matmul(hidden_layer3, weights) + biases
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
    # Optimizer.
    global_step = tf.Variable(0, trainable=False)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.01, global_step, 100000, 0.5)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
      tf.matmul(
          tf.nn.relu(tf.matmul(
          tf.nn.relu(tf.matmul(
          tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights1) + hidden_biases1)
          , hidden_weights2) + hidden_biases2)
          , hidden_weights3) + hidden_biases3)
          , weights) + biases)
    test_prediction = tf.nn.softmax(
      tf.matmul(
          tf.nn.relu(tf.matmul(
          tf.nn.relu(tf.matmul(
          tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights1) + hidden_biases1)
          , hidden_weights2) + hidden_biases2)
          , hidden_weights3) + hidden_biases3)
          , weights) + biases)

In [ ]:
num_steps = 100001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("\nMinibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print(valid_prediction.eval())
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized

Minibatch loss at step 0: 38.665863
Minibatch accuracy: 7.0%
Validation accuracy: 21.6%
[[  1.22797508e-10   8.84943452e-10   2.24076544e-08 ...,   9.64072708e-04
    4.83818186e-09   1.08507893e-05]
 [  1.24976585e-10   5.18571075e-10   2.81640244e-13 ...,   8.74882335e-06
    1.24464545e-08   9.99952197e-01]
 [  1.05357736e-12   6.50306864e-10   4.58491286e-07 ...,   3.07324044e-05
    2.89439413e-07   5.59558392e-01]
 ..., 
 [  2.55114656e-15   4.73291895e-09   8.64278675e-08 ...,   8.99463348e-09
    1.29276188e-04   9.85556424e-01]
 [  3.89379883e-11   1.22651272e-03   1.51320705e-07 ...,   7.20448152e-04
    6.47726210e-08   5.82729044e-05]
 [  1.42863943e-24   4.57323807e-07   1.03078709e-11 ...,   1.74387805e-12
    9.99953747e-01   1.47376795e-08]]

Minibatch loss at step 500: 1.320636
Minibatch accuracy: 71.1%
Validation accuracy: 81.5%
[[  1.63055503e-07   9.99975204e-01   1.57145408e-08 ...,   1.94403196e-08
    1.15420233e-08   1.34765983e-07]
 [  2.75433739e-